In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 1: Setup & Imports

import os
import numpy as np
import cv2
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, io
from segment_anything import SamPredictor, sam_model_registry


In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 2: Load Sample Image

image_path = 'data/raw/spheres.tif'
img = tifffile.imread(image_path)
print("Image shape:", img.shape)

In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 3: Visualize Image

plt.figure(figsize=(6, 6))
plt.imshow(img, cmap='gray')
plt.title("Raw Spheres Image")
plt.axis("off")
plt.show()

In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 4: Load Cellpose & SAM Models

cellpose_model = models.Cellpose(model_type='cyto2')

sam_checkpoint = "sam_vit_h.pth"  # Replace with your local SAM checkpoint path
sam = sam_model_registry["vit_h"](checkpoint=sam_checkpoint)
sam_predictor = SamPredictor(sam)


In [ ]:
# Zack ,Cell Seeding Efficiency Measure -  Cell 5: Apply Segmentation

masks, flows, styles, diams = cellpose_model.eval([img], channels=[0,0], diameter=None)

In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 6: Generate Class-wise Masks

classified_masks = {
    "circular_live": masks[0] == 1,
    "fixed": masks[0] == 2,
    "dead": masks[0] == 3,
    "fragments": masks[0] == 4
}

In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 7: Save Masks to results/masks/

cv2.imwrite(f'results/masks/{label}.png', (mask * 255).astype(np.uint8))

In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 8: Generate Overlay AVI Videos

overlay = cv2.addWeighted(img, 0.7, (masks[0] > 0).astype(np.uint8) * 255, 0.3, 0)
video_out = cv2.VideoWriter('results/videos/overlay.avi', cv2.VideoWriter_fourcc(*'XVID'), 10, (img.shape[1], img.shape[0]), False)
video_out.write(overlay)
video_out.release()

In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 9: Calculate Metrics (Area, Brightness)

metrics = []
labeled_mask = masks[0]
for region_id in np.unique(labeled_mask):
    if region_id == 0:
        continue
    region_mask = labeled_mask == region_id
    brightness = np.mean(img[region_mask])
    area = np.sum(region_mask)
    metrics.append({
        "region": region_id,
        "brightness": brightness,
        "area": area
    })

import pandas as pd
pd.DataFrame(metrics).to_csv("results/metrics.csv", index=False)

In [ ]:
# Zack ,Cell Seeding Efficiency Measure - Cell 10: Wrap-up

print("Segmentation complete. Results saved to results/")